## Import packages

In [96]:
import os
import pandas as pd
import numpy as np
import requests
import json
import time
from datetime import datetime, timedelta

from shroomdk import ShroomDK

## API Set-up

In [21]:
# Get API_KEY from file

def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r', encoding="utf-8-sig") as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

In [22]:
# Get all results from query with pagination

def querying_pagination(query_string):
    sdk = ShroomDK(API_KEY)
    
    # Query results page by page and saves the results in a list
    # If nothing is returned then just stop the loop and start adding the data to the dataframe
    result_list = []
    for i in range(1,11): # max is a million rows @ 100k per page
        data=sdk.query(query_string,page_size=100000,page_number=i)
        if data.run_stats.record_count == 0:  
            break
        else:
            result_list.append(data.records)
        
    # Loops through the returned results and adds into a pandas dataframe
    result_df=pd.DataFrame()
    for idx, each_list in enumerate(result_list):
        if idx == 0:
            result_df=pd.json_normalize(each_list)
        else:
            result_df=pd.concat([result_df, pd.json_normalize(each_list)])

    return result_df

In [23]:
# Call query, save csv and return the df of a query
def query_to_df(sql_query):
    df = querying_pagination(sql_query)
    df.to_csv(f'data/{sql_query}csv', index=False) 
    return df

In [24]:
API_KEY = get_file_contents('/home/agustin/DS/Projects/NearSocialSegmentApp/keys/apikey.txt')

# Initialize `ShroomDK` with my API Key
sdk = ShroomDK(API_KEY)

## SQL Queries with SDK API

In [46]:
# Near Social auths
auths_raw_sql = open('/home/agustin/DS/Projects/NearSocialSegmentApp/data/queries/sn_auths.txt').read()

auths_raw_df = querying_pagination(auths_raw_sql)

auths_raw_df.to_csv('/home/agustin/DS/Projects/NearSocialSegmentApp/data/csv/auths_raw_df.csv', index=False) 

In [94]:
# Near Social DeFi
defi_raw_sql = open('/home/agustin/DS/Projects/NearSocialSegmentApp/data/queries/sn_defi_activity.txt').read()

defi_raw_df = querying_pagination(defi_raw_sql)

defi_raw_df.to_csv('/home/agustin/DS/Projects/NearSocialSegmentApp/data/csv/defi_raw_df.csv', index=False) 

UserError: user error occured with status code: 400, msg: SQL compilation error: Failure during expansion of view 'FACT_ACTIONS_EVENTS_ADDKEY': SQL compilation error: Schema 'NEAR.SILVER' does not exist or not authorized.

In [49]:
# Near Social users
users_raw_sql = open('/home/agustin/DS/Projects/NearSocialSegmentApp/data/queries/sn_users.txt').read()

users_raw_df = querying_pagination(users_raw_sql)

users_raw_df.to_csv('/home/agustin/DS/Projects/NearSocialSegmentApp/data/csv/users_raw_df.csv', index=False) 

In [89]:
auths_raw_df

,user,first_social_sign,wallet_creation,age_days,age_category,human_readable
0,0d385e3d331d4112a761ec89f80d25cb2fc0eef0ca4b67...,2023-03-14,2023-02-21,21,0-3 MONTHS,NO
1,jumpgroupfoundation.near,2023-03-14,2022-10-06,159,3-6 MONTHS,YES
2,sunsheng.near,2023-03-14,2021-05-14,669,1-2 YEARS,YES
3,oliveragustin.near,2023-03-14,2023-03-02,12,0-3 MONTHS,YES
4,meta-artfun.near,2023-03-14,2022-08-28,198,6-12 MONTHS,YES
...,...,...,...,...,...,...
8070,rucommunity.near,2022-02-18,2021-04-06,318,6-12 MONTHS,YES
8071,kotleta.near,2022-02-17,2021-06-01,261,6-12 MONTHS,YES
8072,vlad.near,2022-02-17,2020-07-31,566,1-2 YEARS,YES
8073,aliaksandrh.near,2022-02-17,2020-09-13,522,1-2 YEARS,YES


In [95]:
defi_raw_df

,user,first_social_sign,number_of_swaps,swaps_category,swapped_before_social,number_stake_actions,current_stake_raw,current_stake,stake_category,staked_before_social,defi_category
0,tyty14.near,2023-03-05 09:01:25.754,1,1-10 SWAPS,YES,1,0.128619,0.128619,0-100 STAKED,YES,DEFI MASTER
1,pixel8llc.near,2022-10-06 16:50:38.990,5,1-10 SWAPS,YES,8,-4.743708,0.000000,NO STAKE,YES,DEFI MASTER
2,ilerik.near,2022-05-23 05:05:25.281,0,NO SWAPS,NEVER SWAPPED,7,0.492973,0.492973,0-100 STAKED,YES,ONLY STAKED
3,sonudb54.near,2023-03-05 14:14:22.355,3,1-10 SWAPS,YES,0,NaN,0.000000,NO STAKE,NEVER STAKED,ONLY SWAPPED
4,pam1.near,2023-03-06 08:41:44.904,0,NO SWAPS,NEVER SWAPPED,0,NaN,0.000000,NO STAKE,NEVER STAKED,NO DEFI ACTIVITY
...,...,...,...,...,...,...,...,...,...,...,...
8070,hackerluck.near,2023-03-05 01:56:53.373,0,NO SWAPS,NEVER SWAPPED,0,NaN,0.000000,NO STAKE,NEVER STAKED,NO DEFI ACTIVITY
8071,a1f5411e1497eacb7a394499bc857f6d27da86b53540b6...,2023-02-23 16:23:50.793,10,1-10 SWAPS,YES,0,NaN,0.000000,NO STAKE,NEVER STAKED,ONLY SWAPPED
8072,nallin.near,2022-12-08 18:28:26.881,0,NO SWAPS,NEVER SWAPPED,0,NaN,0.000000,NO STAKE,NEVER STAKED,NO DEFI ACTIVITY
8073,ebonybarker.near,2023-03-11 06:03:08.105,3,1-10 SWAPS,YES,0,NaN,0.000000,NO STAKE,NEVER STAKED,ONLY SWAPPED


## BACKUP